In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import chess
import chess.pgn
import numpy as np
import os
from pathlib import Path



In [4]:
class ChessboardEncoder:
    def __init__(self):
        self.piece_mapping = {
            'P': 1, 'N': 2, 'B': 3, 'R': 4, 'Q': 5, 'K': 6,
            'p': -1, 'n': -2, 'b': -3, 'r': -4, 'q': -5, 'k': -6
        }
    
    def encode_board(self, board):
        encoded = np.zeros((12, 8, 8), dtype=np.float32)
        for i in range(8):
            for j in range(8):
                piece = board.piece_at(chess.square(j, i))
                if piece is not None:
                    piece_symbol = piece.symbol()
                    color_idx = 0 if piece.color else 6
                    piece_type = abs(self.piece_mapping[piece_symbol]) - 1
                    encoded[piece_type + color_idx][i][j] = 1
        return torch.FloatTensor(encoded)

class ChessDataset(Dataset):
    def __init__(self, games):
        self.encoder = ChessboardEncoder()
        self.positions = []
        self.moves = []
        
        for game in games:
            board = game.board()
            for move in game.mainline_moves():
                from_square = move.from_square
                to_square = move.to_square
                move_idx = from_square * 64 + to_square
                self.positions.append(board.copy())
                self.moves.append(move_idx)
                board.push(move)
    
    def __len__(self):
        return len(self.positions)
    
    def __getitem__(self, idx):
        position = self.encoder.encode_board(self.positions[idx])
        move = torch.LongTensor([self.moves[idx]])
        return position, move


In [5]:
class ChessCNN(nn.Module):
    def __init__(self):
        super(ChessCNN, self).__init__()
        self.conv1 = nn.Conv2d(12, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
        self.fc1 = nn.Linear(256 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 4096)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = x.view(-1, 256 * 8 * 8)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def load_chess_data(pgn_file, max_games=100):
    games = []
    with open(pgn_file, encoding='utf-8') as f:
        for _ in range(max_games):
            game = chess.pgn.read_game(f)
            if game is None:
                break
            games.append(game)
    return games

def train_model(model, train_loader, num_epochs=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    optimizer = optim.Adam(model.parameters())
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_idx, (positions, moves) in enumerate(train_loader):
            positions = positions.to(device)
            moves = moves.to(device).squeeze()
            
            optimizer.zero_grad()
            output = model(positions)
            loss = criterion(output, moves)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            
            if batch_idx % 10 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}')
        
        avg_loss = total_loss / len(train_loader)
        print(f'Epoch {epoch} complete - Average Loss: {avg_loss:.4f}')
        
        # Save model
        torch.save(model.state_dict(), f'C:/Users/tsaid/Downloads/chess_model_epoch_{epoch}.pt')
    
    return model

In [4]:
def main():
    # Load data
    games = load_chess_data("C:/Users/tsaid/Downloads/Andreikin.pgn", max_games=100)
    dataset = ChessDataset(games)
    train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Create and train model
    model = ChessCNN()
    trained_model = train_model(model, train_loader,5)
    
    # Save final model
    torch.save(trained_model.state_dict(), 'C:/Users/tsaid/Downloads/chess_model_final.pt')

if __name__ == "__main__":
    main()

Epoch 0, Batch 0, Loss: 8.3194
Epoch 0, Batch 10, Loss: 7.6250
Epoch 0, Batch 20, Loss: 7.4897
Epoch 0, Batch 30, Loss: 7.4828
Epoch 0, Batch 40, Loss: 7.2955
Epoch 0, Batch 50, Loss: 7.2236
Epoch 0, Batch 60, Loss: 7.5117
Epoch 0, Batch 70, Loss: 6.3424
Epoch 0, Batch 80, Loss: 6.5256
Epoch 0, Batch 90, Loss: 6.3139
Epoch 0, Batch 100, Loss: 6.6746
Epoch 0, Batch 110, Loss: 6.2512
Epoch 0, Batch 120, Loss: 6.8604
Epoch 0, Batch 130, Loss: 6.7040
Epoch 0, Batch 140, Loss: 6.2996
Epoch 0, Batch 150, Loss: 6.5553
Epoch 0, Batch 160, Loss: 6.3330
Epoch 0, Batch 170, Loss: 6.5633
Epoch 0, Batch 180, Loss: 6.8273
Epoch 0, Batch 190, Loss: 6.7327
Epoch 0, Batch 200, Loss: 6.2700
Epoch 0, Batch 210, Loss: 6.0164
Epoch 0, Batch 220, Loss: 7.4014
Epoch 0, Batch 230, Loss: 6.1947
Epoch 0 complete - Average Loss: 6.8019
Epoch 1, Batch 0, Loss: 6.4490
Epoch 1, Batch 10, Loss: 6.4223
Epoch 1, Batch 20, Loss: 5.4343
Epoch 1, Batch 30, Loss: 6.2658
Epoch 1, Batch 40, Loss: 6.3811
Epoch 1, Batch 50, L

In [6]:
def load_model(model_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = ChessCNN().to(device)
    model.load_state_dict(torch.load(model_path, weights_only=True))
    model.eval()
    return model

def get_best_move(model, board, encoder):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    state = encoder.encode_board(board).unsqueeze(0).to(device)
    
    with torch.no_grad():
        output = model(state)
        move_probs = torch.softmax(output, dim=1)
        move_idx = move_probs.argmax(dim=1).item()
        
        from_square = move_idx // 64
        to_square = move_idx % 64
        move = chess.Move(from_square, to_square)
        
        if move in board.legal_moves:
            return move
        else:
            return list(board.legal_moves)[0]

def test_model():
    model = load_model('./chess_model_final.pt')
    encoder = ChessboardEncoder()
    board = chess.Board()
    
    for _ in range(5):  # Play 5 moves
        if board.is_game_over():
            break
            
        move = get_best_move(model, board, encoder)
        print(f"Position: {board.fen()}")
        print(f"Model's move: {move}")
        board.push(move)
        print(board)
        print("\n")

if __name__ == "__main__":
    test_model()

Position: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1
Model's move: e2e4
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


Position: rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1
Model's move: e7e6
r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R


Position: rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBNR w KQkq - 0 2
Model's move: g1f3
r n b q k b n r
p p p p . p p p
. . . . p . . .
. . . . . . . .
. . . . P . . .
. . . . . N . .
P P P P . P P P
R N B Q K B . R


Position: rnbqkbnr/pppp1ppp/4p3/8/4P3/5N2/PPPP1PPP/RNBQKB1R b KQkq - 1 2
Model's move: g8f6
r n b q k b . r
p p p p . p p p
. . . . p n . .
. . . . . . . .
. . . . P . . .
. . . . . N . .
P P P P . P P P
R N B Q K B . R


Position: rnbqkb1r/pppp1ppp/4pn2/8/4P3/5N2/PPPP1PPP/RNBQKB1R w KQkq - 2 3
Model's move: f3e5
r n b q k b . r
p p p p . p p p